In [63]:
import numpy as np
import pandas as pd
from utils import process_data, get_device
import torch

In [93]:
df = pd.read_csv("dataset/BA_weather_6-23_jan.csv", parse_dates=["Date"])

In [94]:
weather_data = df.iloc[::-1]
weather_data.reset_index(drop=True, inplace=True)


In [95]:
weather_data

,Date,T,P0,U,Ff
0,2023-06-01 00:00:00,9,753.9,71,4
1,2023-06-01 00:30:00,9,753.9,71,4
2,2023-06-01 01:00:00,9,753.9,71,4
3,2023-06-01 01:30:00,8,754.7,76,4
4,2023-06-01 02:00:00,8,754.7,76,3
...,...,...,...,...,...
857,2023-01-23 21:30:00,2,759.6,87,3
858,2023-01-23 22:00:00,1,759.6,100,3
859,2023-01-23 22:30:00,1,760.4,93,3
860,2023-01-23 23:00:00,2,760.4,87,2


In [96]:
X, y = process_data(weather_data)

/home/maco/Documents/git-repos/WeatherProphet/utils.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data["day_of_year"] = weather_data["Date"].dt.dayofyear
/home/maco/Documents/git-repos/WeatherProphet/utils.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_data["time"] = pd.to_datetime(
/home/maco/Documents/git-repos/WeatherProphet/utils.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [97]:
mean, std = np.load("dataset/stat.npy")

In [98]:
X = (X - mean) / std

In [99]:
X

,sin_day_of_year,cos_day_of_year,sin_hour,cos_hour,P0,T,U,Ff
0,0.724488,-1.241554,-0.000427,1.415490,0.544631,-0.296610,-0.033131,0.110761
2,0.724488,-1.241554,0.365557,1.367297,0.544631,-0.296610,-0.033131,0.110761
4,0.724488,-1.241554,0.706600,1.226002,0.683916,-0.409101,0.243570,-0.334331
6,0.724488,-1.241554,0.999460,1.001235,0.683916,-0.296610,-0.033131,-0.334331
8,0.724488,-1.241554,1.224180,0.708312,0.683916,-0.409101,0.243570,-0.334331
...,...,...,...,...,...,...,...,...
852,0.560688,1.282359,-1.366298,0.367197,1.554444,-0.971558,0.852314,-0.779423
854,0.560688,1.282359,-1.225034,0.708312,1.554444,-0.971558,0.852314,-0.779423
856,0.560688,1.282359,-1.000314,1.001235,1.537033,-1.196540,1.184355,-0.334331
858,0.560688,1.282359,-0.707454,1.226002,1.537033,-1.196540,1.571737,-0.334331


In [100]:
y

0      9.0
2      8.0
4      9.0
6      8.0
8      8.0
      ... 
852    3.0
854    1.0
856    1.0
858    2.0
860    NaN
Name: next_T, Length: 431, dtype: float64

In [101]:
X, y = X.to_numpy()[:-1], y.to_numpy()[:-1]

In [102]:
a = []
b = []
for i in range(12):
    idx = 120+i
    a.append(X[idx: idx+6])
    b.append(y[idx + 6])

In [103]:
y1 = np.asarray(b)

In [104]:
y1.shape

(12,)

In [105]:
x1 = np.asarray(a)

In [106]:
x1.shape

(12, 6, 8)

In [108]:
device = get_device()


In [109]:
wp = torch.load('WP100-wtf_multistep3.pt')

In [110]:
wp.to(device)

ClassicWeatherProphet(
  (model): Sequential(
    (0): Linear(in_features=48, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [111]:
wp.eval()

ClassicWeatherProphet(
  (model): Sequential(
    (0): Linear(in_features=48, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [112]:
X_features = torch.from_numpy(x1).float().to(device)

In [113]:
X_features.shape

torch.Size([12, 6, 8])

In [114]:
y_hat = wp(X_features)

In [115]:
y_hat

tensor([[[2.0000]],

        [[2.0091]],

        [[2.0433]],

        [[1.9302]],

        [[3.0050]],

        [[6.0111]],

        [[6.9826]],

        [[9.0112]],

        [[8.9930]],

        [[9.0057]],

        [[7.9988]],

        [[6.0246]]], device='cuda:0', grad_fn=<ViewBackward0>)

In [116]:
y

array([ 9.,  8.,  9.,  8.,  8.,  8.,  7.,  6.,  8.,  8.,  9., 10., 10.,
       10., 10.,  9.,  8.,  7.,  8.,  7.,  9., 10., 10., 10., 10., 10.,
        9.,  9.,  7.,  9.,  8.,  8.,  6.,  9.,  9.,  9., 10., 10., 10.,
        7.,  6.,  5.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  7.,  7.,  7.,  7.,  5.,
        5.,  5.,  3.,  2.,  0.,  1.,  1.,  1.,  1.,  2.,  3.,  4.,  5.,
        6.,  9.,  9.,  9.,  9.,  9.,  8.,  8.,  7.,  7.,  7.,  7.,  7.,
        7.,  7.,  7.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.,
        6.,  6.,  6.,  7.,  7.,  7.,  8.,  8.,  7.,  7.,  7.,  7.,  7.,
        6.,  6.,  6.,  5.,  5.,  5.,  4.,  2.,  2.,  2.,  2.,  3.,  6.,
        7.,  9.,  9.,  9.,  8.,  6.,  6.,  5.,  4.,  4.,  4.,  5.,  4.,
        4.,  4.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  6.,  9., 11.,
       11., 12., 11.,  8.,  8.,  8.,  7.,  5.,  4.,  4.,  4.,  3.,  3.,
        3.,  2.,  2.,  2.,  1.,  1.,  2.,  3.,  3.,  4.,  5.,  6